# Data Engineering: Preprocessing & Merging data from multiple sources

The identification of appropriate sources of data and its pre-processing is the integral part of any data analytics task. <font color="blue">Data wrangling</font> accounts for the most tedious and time-consuming activity in the analytics pipeline. Here, we try to collect data about *playing pitches in area around Dublin, Ireland* in various <font color="brown">heterogenous formats</font> like CSV/ XML/ KML data files and REST webservices (for geographical data).     

**Data Source:** [Open data from Irish government portal](https://data.gov.ie/)    

<b><font color="blue" size=3>Objective:</font></b>    

The aim of this task is to create a dataset that exhibits following properties:   

1.	**Complete:** Should include all relevant fields from individual dataset
2.	**Clean:** Free of discrepancies
3.	**Unique:** No duplicate records
4.	**Consistent:** Field values should be represented uniformly     

<b><font color="blue" size=3>Key Tasks performed:</font></b>    
1. **Exploratory Analysis:** Understanding data format and issues  
2. **Data Modelling:** Designing final unified data model    
3. **Data Quality Enhancement:**   
    3.1. *Data cleaning:* Handle erroneous data, outliers, duplicates and other such issues    
    3.2. *Data enrichment:* Obtain missing geographical data using geocoding from the available REST web-services   
4. **Merging:** Combine data from all the sources into a single structure that could be used for further analysis.  

<b><font color="blue" size=3>Important libraries used:</font></b>  
1. Pandas: For storing and manipulating the data in structured data frames    
2. Numpy: For statistical analysis of data    
3. ArcGIS and Geopy API for accessing related geographical web services

In [82]:
#Importing prerequisite packages
import pandas as pd
from arcgis.gis import *
from arcgis.geocoding import geocode, reverse_geocode
from arcgis.geometry import Point
import math
from geopy.geocoders import Nominatim
import re
import xml.etree.ElementTree as et
import numpy as np
#Initialize GIS objects
geolocator = Nominatim()
gis_obj = GIS()

### Import data from three datasets for observatory analysis

In [83]:
def read_XML(xml_path):
    '''Function to read data from XML files to a pandas dataframe'''
    parse_XML = et.parse(xml_path)
    root = parse_XML.getroot()
    facility_names = []
    facility_types = []
    locations = []
    latitudes = []
    longitudes = []
    for pitch in root.find('Playing_Pitches-table').findall('*'):
        facility_names.append(pitch.find('FACILITY_NAME').text)
        facility_types.append(pitch.find('FACILITY_TYPE').text)
        locations.append('NA' if not pitch.find('LOCATION').text else pitch.find('LOCATION').text)
        latitudes.append(pitch.find('LAT').text)
        longitudes.append(pitch.find('LONG').text)
    df = pd.DataFrame(
        {
            'FACILITY_NAME': facility_names,
            'FACILITY_TYPE': facility_types,
            'LOCATION': locations,
            'LAT': latitudes,
            'LONG': longitudes,
        }
    )
    return df[['FACILITY_NAME', 'FACILITY_TYPE', 'LOCATION', 'LAT', 'LONG']]

In [84]:
df_dcc = pd.read_csv('dccplayingpitchesp20120816-1550.csv') #Read from CSV
df_fcc = read_XML('fccplayingpitchesp20111203-1424.xml') #Read from XML
df_dlr = pd.read_csv('dlr-pitches.csv') #Read from CSV

In [77]:
pd.set_option("display.max_rows", 10)

### Data Observation
 First, I performed an exploratory analysis over each of the 3 datasets and made the following observations:
 <br><br>1. Playing pitches in Dublin City Council (<strong>DCC dataset</strong>):  

In [86]:
df_dcc.head(10)

,PARK,AREA,CLUBNAME,LEAGUE,Unnamed: 4
0,ALBERT COLLEGE,NORTH WEST,DRUMCONDRA F.C (Snr),AMATEUR FOOTBALL LEAGUE,NaN
1,ALBERT COLLEGE,NORTH WEST,GLASNEVIN AFC,AMATEUR FOOTBALL LEAGUE,NaN
2,BEECHILL,SOUTH EAST,BALLSBRIDGE FC,AMATEUR FOOTBALL LEAGUE,NaN
3,BELCAMP,NORTH CENTRAL,NEWTOWN CELTIC,AMATEUR FOOTBALL LEAGUE,NaN
4,BELCAMP,NORTH CENTRAL,VIANNEY BOYS,AMATEUR FOOTBALL LEAGUE,NaN
5,BELCAMP,NORTH CENTRAL,PRIORSWOOD F.C SENIORS,AMATEUR FOOTBALL LEAGUE,NaN
6,BELCAMP,NORTH CENRAL,WHITES F.C,AMATEUR FOOTBALL LEAGUE,NaN
7,BELCAMP,NORTH CENTRAL,URBAN CELTIC,AMATEUR FOOTBALL LEAGUE,NaN
8,BELCAMP,NORTH CENTRAL,CLONSHAUGH SPORTING SUAZEE,AMATEUR FOOTBALL LEAGUE,NaN
9,DARNDALE,NORTH CENTRAL,J.K. CELTIC,AMATEUR FOOTBALL LEAGUE,NaN


In [91]:
#Explore number of observations and unique values for all columns
df_dcc.iloc[:,0:4].describe()

,PARK,AREA,CLUBNAME,LEAGUE
count,250,250,250,250
unique,90,9,205,31
top,ST ANNES,NORTH CENTRAL,MARINO A.F.C.,GAELIC ATHLETIC ASSOCIATION
freq,19,98,3,37


In [93]:
sum(df_dcc.iloc[:,0:4].isnull().any(axis=1)) #Check if any of the row has any value missing (apart from last column)

0

As seen above, the dataset contains 250 records, of which 90 <i>seem</i> to be unique. Also, no field data is missing.<br>
<font color='blue'><br><strong>Issues with DCC dataset:</strong></font>
1. <i>Column Headers and park, club, league names in upper case:</i><br> Need to change to title case
2. <i>Discrepancies in Park names (spaces, periods, eg. "St.  Annes", " St Annes "):</i>
    <br>Strip spaces trailing, following  the names and replace extra spaces in between the names with single space. Remove periods.
3. Extra column of null values due to discrepancy in CSV file. Need to import only required columns

**(2) Playing pitches in Dún Laoghaire-Rathdown county council (<font color = "green">DLR dataset</font>): **

In [94]:
sum(df_dlr.isnull().any(axis=1)) #Check number of rows missing any values

49

In [95]:
df_dlr.head(25)

,Location,Number,Size,Latitude,Longitude
0,Kilbogget Park,1,Snr,53.257242,-6.140665
1,NaN,2,SSG,53.257614,-6.139882
2,NaN,3,SSG,53.257842,-6.139265
3,NaN,4,SSG,53.257098,-6.139094
4,NaN,5,SGG,53.256674,-6.140134
5,NaN,6,Snr,53.256597,-6.138149
6,NaN,7,Snr,53.256238,-6.138761
7,NaN,8,Snr,53.256052,-6.139716
8,NaN,9,Snr,53.255667,-6.141805
9,NaN,10,SSG,53.258577,-6.142454


In [89]:
#Find unique values of Pitch number
df_dlr.Number.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', 'A', 'B',
       'All weather', '12', '13', '14', 'GAA A', 'GAA B', 'Soccer', 'GAA',
       'Temporary pitch', 'Rugby', ' GAA ', 'Location of future pitchs',
       'A ', nan], dtype=object)

In [90]:
#Find unique values of Pitch size
df_dlr.Size.unique()

array(['Snr', 'SSG', 'SGG', '11', 'SNR', 'JNR', 'Mini', 'Full', nan,
       'mini', ' Snr '], dtype=object)

In [97]:
#Check rows with missing values (apart from obvious location names)
df_dlr[df_dlr.iloc[:,1:4].isnull().any(axis=1)] 

,Location,Number,Size,Latitude,Longitude
28,Shanganagh Castle,Temporary pitch,NaN,NaN,NaN
55,Hundson Road,Location of future pitchs,NaN,53.284954,-6.121879
60,Mount Albany,NaN,NaN,53.286265,-6.173321


<font color='blue'><br><strong>Issues with DLR dataset:</strong></font>
1. <i>Missing Location (Park) names:</i><br> Fill missing location names with previous values
2. <i>Inconsistent pitch size values:</i>
    <br>Replace pitch size abbreviations with actual sizes. eg. SNR with 'Senior'  
3. Missing Latitude/ Longitude for one of the observations (Shanganagh Castle)
4. 'SGG' is an incorrectly recorded pitch type for a single observation. Should be changed to 'Single Sided Games'

**(3) Playing pitches in Fingal county council (<font color = "green">FCC dataset</font>): **

In [94]:
df_fcc.head()

,FACILITY_TYPE,FACILITY_NAME,LOCATION,LAT,LONG
0,All weather pitches,Balbriggan Town Park,Balbriggan,53.604960,-6.182353
1,All weather pitches,Balheary Reservoir,Swords,53.472710,-6.223015
2,All weather pitches,Town Park,Skerries,53.577114,-6.111072
3,All weather pitches,St. Mologa's Park,Balbriggan,53.617667,-6.189368
4,Basketball Court,Seagrange Park,NaN,53.396667,-6.135352


In [95]:
#Check rows with missing values
df_fcc[df_fcc.isnull().any(axis=1)] 

,FACILITY_TYPE,FACILITY_NAME,LOCATION,LAT,LONG
4,Basketball Court,Seagrange Park,NaN,53.396667,-6.135352
5,Basketball Court,Grace O'Malley Park,NaN,53.383767,-6.069940
13,GAA Pitches,Hartstown Park,NaN,53.395924,-6.411545
14,GAA Pitches,Hartstown Park,NaN,53.395323,-6.412928
19,GAA Pitches,Seagrange Park,NaN,53.393783,-6.136071
...,...,...,...,...,...
85,Soccer pitches,Seagrange Park,NaN,53.392369,-6.134787
87,Soccer pitches,St. Catherines,NaN,53.366758,-6.470434
88,Soccer pitches,St. Catherines,NaN,53.366696,-6.469224
89,Soccer pitches,St. Catherines,NaN,53.366119,-6.464201


<font color='blue'><br><strong>Issues with FCC dataset:</strong></font>
1. <i>Missing Location names:</i><br> Fetch missing location values using location co-ordinates and reverse geocoding
2. <i>All upper case headers:</i>
    <br>Replace headers with title case and use full names for latitude/ longitude column headers

### Data Modelling

The modelling objective is to create a clean and complete dataset of playing pitches around Dublin. All the datasets have different set of features and we need to include only the relevant ones. In alignment to our objective, I identify the need to include the following features in the resultant dataset:
1. 'Park': Park or facility name, where the pitches are located
2. 'Location': Brief address of the park
3. 'Latitude': X location co-ordinate
4. 'Longitude': Y location co-ordinate
5. 'County Council': one of the 3 county councils in consideration
6. 'Additional Information': Miscellaneous information, that is unique in each dataset. eg. Pitch Number, Size, Clubs, type of pitch

<br>Resultant dataset would have the following schema:</br>

| Park         | Location | Latitude  | Longitude | County Council | Additional Information |
|:------------:|:--------:|:---------:|:---------:|:--------------:|:----------------------:|


### Data Quality Enhancement

In order to achieve the above resultant data model, each of the 3 datasets must be wrangled individually.

<i><u>Note regarding geo-spatial information retrieval techniques used:</u></i>
<br>The process of geocoding and reverse geocoding has been employed to enhance the data quality of all three datasets. A combination of two APIs is used to fetch the geospatial data:
1. <font color='blue'>Nominatim</font> with geopy: 
 - Used to search <i>Open Street Maps</i> data
 - OpenStreetMap® is open data, licensed under the Open Data Commons Open Database License (ODbL) by the OpenStreetMap Foundation (OSMF).
 - Fairly accurate and updated cartography data
 - users are free to copy, distribute, transmit and adapt the data (unlike some other proprietary services), as long as you credit OpenStreetMap and its contributors

2. <font color='blue'>ArcGIS</font> for Developers API with ArcGIS python package:
 - provided by  ESRI (Environmental Systems Research Institute) which is an international supplier of geographic information system (GIS) software, web GIS and geodatabase management applications 
 - High degree of accuracy over smaller area of interest

##ArcGIS service has been availed only for locations whose details are unavailable with Nominatim


In [98]:
def find_location_coordinates_DCC(row, previous_park, previous_location):
    '''Searches Lat-Long and location of a park'''
    park = row['Park'].strip()
    #Check if geocoding has been done previously for the same location 
    if park != previous_park:
        #Request geocoding details using geopy
        #If location coordinates are found by geopy, try with Arcgis
        location_arcgis = geocode(address={"Address" : park, 'Region':'Dublin'}, as_featureset=True)
        if location_arcgis is not None:
            row['Latitude'] = location_arcgis.features[0].geometry.y
            row['Longitude'] = location_arcgis.features[0].geometry.x
            row['Location'] = location_arcgis.features[0].attributes['Nbrhd']
        else:
        #If location not found either by geopy or arcgis, set NaN
            row['Latitude'] = math.nan
            row['Longitude'] = math.nan
        
         #Remember current location for next row
        previous_location['Latitude'] = row['Latitude']
        previous_location['Longitude'] = row['Longitude']
        previous_location['Location'] = row['Location']
    else:
        #if location details were previously queried, assign those instead of querying again
        row['Latitude'] = previous_location['Latitude']
        row['Longitude'] = previous_location['Longitude']
        row['Location'] = previous_location['Location']
    previous_park = park #Remember current park name for next row
    return row

**(1) Dublin City Council (DCC) dataset:**

<font color="green">(A) Data Cleaning:</font>
1. Update all headers and column values to title case
2. Strip spaces trailing and following the park names. Replace extra spaces in between the names with single space. Remove periods in names like Fr., St.

<br><font color="green">(B) Data Enhancement:</font>
1. Retrieve park location, **Latitude and Longitude co-ordinates** using Park names and geocoding
2. **Group** records by park X, Y location co-ordinates and aggregate club names in a single comma separated value, so that duplicate par records are exluded. A park associated with multiple clubs had multiple redundant entries in the dataset. Now, all clubs associated with single park would be combined together

<u><i><b>Note:</b></i></u> Data cleansing and data enhancement is elegantly achieved by constructing a pipeline of those operations using pandas method chaining

In [106]:
def read_dcc(fp_dcc):
    '''Function to read DCC CSV file and perform pipeline of cleaning and enhancement operations'''
    #Regex pattern to eliminate unnecessary periods and extra spaces in park names
    regex_pattern = re.compile('\.*\s\s*')
    #Initializing previous location and park name variables
    previous_location = {'Latitude': math.nan, 'Longitude':math.nan}
    previous_park = ''
    
    df_dcc = (pd.read_csv(fp_dcc, usecols=['PARK','AREA','CLUBNAME']) #Read only relevant columns from CSV
            .rename(columns=str.title) #Change column header cases
              #Find the location coordinates for the parks
            .pipe(lambda x: x.apply(find_location_coordinates_DCC, axis = 1, previous_park = previous_park, \
                  previous_location = previous_location))
              #Assign citycouncil name to all rows, clean park names and club name values
            .assign(City_Council = 'Dublin', \
                     Clubname = lambda x: x['Clubname'].str.title(),
                       Park = lambda x: x['Park'].str.title().str.strip().replace(regex_pattern, ' ', regex=True))
              #Group all clubs associated with same parks (club names joined separated by comma)
            .groupby(['Park','Latitude', 'Longitude', 'Location', 'City_Council'])['Clubname'].apply(', '.join).reset_index())
    return df_dcc
    
#Save pitches dataset, so that geolocation data retrieval and preprocessing need not be done again
saved_data = 'data/dcc_pitches.h5'

#Check if dataset is saved earleir, if yes, retrieved from saved copy than preprocessing again
if not os.path.exists(saved_data):
    df_dcc = read_dcc("dccplayingpitchesp20120816-1550.csv")
    df_dcc.to_hdf(saved_data, 'dcc_pitches', format='table') #Save to hdf file after processing for the first time
else:
    df_dcc = pd.read_hdf(saved_data, 'dcc_pitches', format='table')

df_dcc.head()

,Park,Latitude,Longitude,Location,City_Council,Clubname
0,Albert College,53.386324,-6.262421,Dublin 9,Dublin,"Drumcondra F.C (Snr), Glasnevin Afc, Greenfiel..."
1,Alfie Byrne,53.361104,-6.227940,Dublin 3,Dublin,"Eastwall Bessborough Utd., Sheriff Y.C."
2,Ardmore,53.424647,-6.348230,Dublin 11,Dublin,Artane/Beaumont Juveniles
3,Ashington,53.371376,-6.318234,Dublin 7,Dublin,Navan Road United
4,Balcurris Park,53.400240,-6.266570,Dublin 11,Dublin,"Setanta, Unidare Rfc"


**(2) Fingal County Council (FCC) dataset:**

<br><font color="green">(A)</font> Import data from <font color="green">XML</font> source:
 - Fingal county council pitch data is imported from XML source using xml package

In [100]:
def read_XML(xml_path):
    '''Function to parse XML data file and retrive Fingal pitches data'''
    parse_XML = et.parse(xml_path)
    root = parse_XML.getroot()
    facility_names = []
    facility_types = []
    locations = []
    latitudes = []
    longitudes = []
    for pitch in root.find('Playing_Pitches-table').findall('*'):
        facility_names.append(pitch.find('FACILITY_NAME').text)
        facility_types.append(pitch.find('FACILITY_TYPE').text)
        locations.append('NA' if not pitch.find('LOCATION').text else pitch.find('LOCATION').text)
        latitudes.append(pitch.find('LAT').text)
        longitudes.append(pitch.find('LONG').text)
    df = pd.DataFrame(
        {
            'FACILITY_NAME': facility_names,
            'FACILITY_TYPE': facility_types,
            'LOCATION': locations,
            'LAT': latitudes,
            'LONG': longitudes,
        }
    )
    return df[['FACILITY_NAME', 'FACILITY_TYPE', 'LOCATION', 'LAT', 'LONG']]

<br><font color="green">(B) Data Cleaning:</font>
1. Update all headers to title case
2. Modify column headers 'LAT' to 'Latitude' and 'LONG' to Longitude
3. Locations in some of the observations are missing. The locations would be fetched from latitude/ longitude coordinates using reverse geocoding.

<br><font color="green">(C) Data Enhancement:</font>
1. A column identifying the county council that the pitches belong to, is added.

In [101]:
def find_location_Fingal(row):
    '''Function that reverse geocodes location from Latituded-Longitude co-ordinates'''
    #Query Location info only if not available
    if row['LOCATION'] == 'NA':
        location = geolocator.reverse(str(row['LAT']) + ", " + str(row['LONG']))
        if location is not None:
            row['LOCATION'] = location[0].split(', Fingal,')[0].split(',')[-1].strip()
    return row

In [102]:

def read_fcc(fp_fcc):
    '''Function to read Fingal dataset from XML file and then pre-process it'''
    df = (read_XML(fp_fcc)
              #Find missing locations
             .pipe(lambda x: x.apply(find_location_Fingal, axis = 1))
              #Assign same city council names to all observations
             .assign(City_Council = 'Fingal')
              #Clean column headers
             .rename(index=str, columns={"FACILITY_NAME": "Park", "LAT": "Latitude", "LONG": "Longitude",
                                        "FACILITY_TYPE":"Facility_Type", "LOCATION":"Location"}))              
    return df
    

saved_fcc_data = 'data/fcc_pitches.h5'

if not os.path.exists(saved_fcc_data):
    df_fcc = read_fcc("fccplayingpitchesp20111203-1424.xml")
    df_fcc.to_hdf(saved_fcc_data, 'fcc_pitches', format='table')
else:
    df_fcc = pd.read_hdf(saved_fcc_data, 'fcc_pitches', format='table')

df_fcc.head()

,Park,Facility_Type,Location,Latitude,Longitude,City_Council
0,Balbriggan Town Park,All weather pitches,Balbriggan,53.6049596246817,-6.18235291959051,Fingal
1,Balheary Reservoir,All weather pitches,Swords,53.4727096370551,-6.22301521551813,Fingal
2,Town Park,All weather pitches,Skerries,53.5771135903791,-6.11107205744599,Fingal
3,St. Mologa's Park,All weather pitches,Balbriggan,53.6176672458903,-6.18936794084573,Fingal
4,Seagrange Park,Basketball Court,Dublin 13,53.3966674985382,-6.13535180348378,Fingal


In [15]:
df_fcc[df_fcc.isnull().any(axis=1)] #Check if any values are still missing

,Park,Facility_Type,Location,Latitude,Longitude,City_Council


Thus, dataset for pitches in Fingal County Council is processed as per the requirements.

**(3) Dún Laoghaire-Rathdown County Council (DCC) dataset:**<br>
<br><font color="green">(A) Data Cleaning:</font>
1. Rename 'Location' column to 'Park' in order to be consistent with proposed data model
2. Fill missing park names with previous values
3. Replace pitch size abbreviations with actual sizes. eg. SNR with 'Senior', 11 with '11-a-side'
4. Fetch missing Latitude/ Longitude for one of the parks (Shanganagh Castle)
5. 'SGG' is an incorrectly recorded pitch type for a single observation. It is changed to 'Single Sided Games'

<br><font color="green">(B) Data Enhancement:</font>
1. A column identifying the *county council* that the pitches belong to, is to be added.
2. A column for park *location* is to be added which would be helpful for completeness. The locations would be fetched from latitude/ longitude coordinates using reverse geocoding. 

In [103]:
def find_address_DLR(row, previous_park, previous_location):
    '''Function to find location of the parks. Also find missing lat-long coordinates'''
    park = row['Park'].strip()
    #Check if geocoding has been done previously for the same location 
    if park != previous_park:
        #Request geocoding details using Arcgis
        location_arcgis = geocode(address={"Address" : park, 'Region':'Dublin'}, as_featureset=True)
        if location_arcgis is not None:
            #Get neighbourhood address as location of the park
            row['Location'] = location_arcgis.features[0].attributes['Nbrhd']
            #Find missing lat long
            if math.isnan(row['Latitude']) or math.isnan(row['Longitude']):
                row['Latitude'] = location_arcgis.features[0].attributes['Y']
                row['Longitude'] = location_arcgis.features[0].attributes['X']
         #Remember current location for next row
        previous_location = row['Location']
    else:
        #if location details were previously queried, assign those instead of querying again
        row['Location'] = previous_location['Location']
    previous_park = park #Remember current park name for next row
    return row

def read_dlr(fp_dlr):
    '''Function to read the pitches data for DLR county council'''
    previous_park = ''
    previous_location = ''
    df = (pd.read_csv(fp_dlr)
              #Clean column headers
            .rename(index=str, columns={"Location": "Park", "Number": "Pitch_Number"}) 
              #Assign common county council name to all observations,
              #Fill missing park names with previous values
              #Update pitch sizes with appropriate full forms to avoid ambiguity
            .assign(City_Council = 'Dún Laoghaire-Rathdown', Park = lambda x: x['Park'].ffill(),
                   Size = lambda x: np.where(x['Size'].str.lower() == 'snr', 'Senior', 
                          np.where(x['Size'] == '11', '11-a-side',
                          np.where(x['Size'].str.lower() == 'ssg', 'Small Sided Games', 
                          np.where(x['Size'].str.lower() == 'sgg', 'Small Sided Games', 
                          np.where(x['Size'].str.lower() == 'jnr', 'Junior',
                          np.where(x['Size'].str.lower() == 'mini', 'Mini',
                          x['Size'])))))))
              #Find locations and missing lat-long for parks
            .pipe(lambda x: x.apply(find_address_DLR, axis = 1, previous_park = previous_park,
                                    previous_location = previous_location)))
                         
    return df
    

saved_dlr_data = 'data/dlr_pitches.h5'

if not os.path.exists(saved_dlr_data):
    df_dlr = read_dlr("dlr-pitches.csv")
    df_dlr.to_hdf(saved_dlr_data, 'dlr_pitches', format='table')
else:
    df_dlr = pd.read_hdf(saved_dlr_data, 'dlr_pitches', format='table')

df_dlr.head(30)

,Park,Pitch_Number,Size,Latitude,Longitude,City_Council,Location
0,Kilbogget Park,1,Senior,53.257242,-6.140665,Dún Laoghaire-Rathdown,Dublin 18
1,Kilbogget Park,2,Small Sided Games,53.257614,-6.139882,Dún Laoghaire-Rathdown,Dublin 18
2,Kilbogget Park,3,Small Sided Games,53.257842,-6.139265,Dún Laoghaire-Rathdown,Dublin 18
3,Kilbogget Park,4,Small Sided Games,53.257098,-6.139094,Dún Laoghaire-Rathdown,Dublin 18
4,Kilbogget Park,5,Small Sided Games,53.256674,-6.140134,Dún Laoghaire-Rathdown,Dublin 18
5,Kilbogget Park,6,Senior,53.256597,-6.138149,Dún Laoghaire-Rathdown,Dublin 18
6,Kilbogget Park,7,Senior,53.256238,-6.138761,Dún Laoghaire-Rathdown,Dublin 18
7,Kilbogget Park,8,Senior,53.256052,-6.139716,Dún Laoghaire-Rathdown,Dublin 18
8,Kilbogget Park,9,Senior,53.255667,-6.141805,Dún Laoghaire-Rathdown,Dublin 18
9,Kilbogget Park,10,Small Sided Games,53.258577,-6.142454,Dún Laoghaire-Rathdown,Dublin 18


In [80]:
#Check if any rows are missing any key values
df_dlr[df_dlr.isnull().any(axis=1)]

,Park,Pitch_Number,Size,Latitude,Longitude,City_Council,Location
28,Shanganagh Castle,Temporary pitch,NaN,53.279036,-6.120960,Dún Laoghaire-Rathdown,Glenageary
55,Hundson Road,Location of future pitchs,NaN,53.284954,-6.121879,Dún Laoghaire-Rathdown,Glasthule
60,Mount Albany,NaN,NaN,53.286265,-6.173321,Dún Laoghaire-Rathdown,Blackrock


Only missing data is pitch number and sizes for few pitches which cannot be retrieved without additional observations. So, DLR dataset is as clean as it can be.

### Combining all the three datasets
Now, we append all the 3 cleaned datasets in order to obtain the  data model proposed earlier.

In [129]:
#Append all 3 datasets one after other
df_pitches = df_dcc.append(df_fcc).append(df_dlr)
df_pitches.loc[:,['Park', 'Location','Latitude', 'Longitude', 
                  'City_Council', 'Facility_Type', 'Pitch_Number', 
                  'Size', 'Clubname']].sample(5)

,Park,Location,Latitude,Longitude,City_Council,Facility_Type,Pitch_Number,Size,Clubname
58,Loughlinstown Park,Loughlinstown,53.2442,-6.1254,Dún Laoghaire-Rathdown,NaN,1,Senior,NaN
55,Hundson Road,Glasthule,53.285,-6.12188,Dún Laoghaire-Rathdown,NaN,Location of future pitchs,NaN,NaN
26,Shanganagh Cliffs,Dublin 18,53.2415,-6.1137,Dún Laoghaire-Rathdown,NaN,Soccer,Senior,NaN
45,Ball Alley/Remount,Lusk,53.5251287367783,-6.16069534107133,Fingal,Soccer pitches,NaN,NaN,NaN
57,Thomastown Road,Glenageary,53.2714,-6.13538,Dún Laoghaire-Rathdown,NaN,B,Senior,NaN


In [116]:
#Check number of rows duplicated in the dataset
sum(df_pitches.duplicated())

0

The combined dataset obtained above, exhibits the following properties:
1. Accurate: As accurate as the underlying open data sources are
2. Unique: No duplicate records
3. Consistent: All variables are consistently represented

**However**, the dataset is not <font color="blue"><i>complete</i></font>. In the sense, it misses values that are characteristic to only individual datasets, like clubnames in Dublin city council, facility types in DLR dataset or pitch number level granularity in Fingal dataset. But, I have included all these fields as they are peculiar of typical playing pitches.
<br>Even as there appear to be a lot of mssing values in columns like clubnames, facility type, etc. the dataset is <strong><font color = "red">tidy</font></strong>. Being a tidy set, it is easier and faster to carry out any further analysis using this structure. As only a single variable is described by a single column and every row has unique observation.

<br>Loosing <i>tidyness</i> but achieving <i>completeness</i>:
 - We can choose to compromise on tidyness of data and still not loose out on any information by combining non-common column into a single column such as 'Additional Information'

In [118]:
df_pitches_complete = (df_dcc.assign(Additional_Details=lambda x: 'Club: '+ x['Clubname'])
                             .drop(['Clubname'], axis = 1))\
                        .append((df_fcc.assign(Additional_Details =lambda x: 'Facility Type: '+ x['Facility_Type'])
                                       .drop(['Facility_Type'], axis = 1)))\
                        .append((df_dlr.assign(Additional_Details =lambda x: 'Pitch Number: '+ x['Pitch_Number'] \
                                               +'| Size: '+ x['Size']))
                                       .drop(['Pitch_Number', 'Size'],axis=1))

In [125]:
#Sampling 10 random rows from the dataset
df_pitches_complete.loc[:,['Park', 'Location','Latitude', 'Longitude', 
                  'City_Council', 'Additional_Details']].sample(10)

,Park,Location,Latitude,Longitude,City_Council,Additional_Details
34,Hartstown Park,Clonsilla,53.3962424394402,-6.41037074601988,Fingal,Facility Type: Seven-a-side
46,Pearse Park,Sallynoggin,53.2751,-6.14209,Dublin,"Club: Crumlin United, Crumlin Hf&C, Good Couns..."
68,Town Park,Balbriggan,53.5756675954932,-6.1079062121554,Fingal,Facility Type: Soccer pitches
32,Hartstown Park,Clonsilla,53.393968080238,-6.41211092873122,Fingal,Facility Type: Seven-a-side
44,Balbriggan Town Park,Balbriggan,53.604642797328,-6.18133609646353,Fingal,Facility Type: Soccer pitches
0,Kilbogget Park,Dublin 18,53.2572,-6.14067,Dún Laoghaire-Rathdown,Pitch Number: 1| Size: Senior
57,Streamville,Killiney,53.2475,-6.12006,Dublin,Club: Clare Hall Celtic
40,Mellowes,Dublin 11,53.3915,-6.31483,Dublin,"Club: Mckelvey Celtic, Finglas Wanderers F.C, ..."
32,Kildonan,Dublin 11,53.3915,-6.30896,Dublin,"Club: W.F.T.A. F.C. Snr., W.F.T.A. S/Boys F.C..."
39,Bayside School,Sutton,53.3897700368883,-6.14914963464,Fingal,Facility Type: Seven-a-side


The above dataset is complete and includes a column of additional details which has peculiar attributes from all 3 datasets. Pitch number and size have been separated by '|'. Analysis over this dataset would be slighlty more difficult than the tidy dataset but this achieves a tradeoff for the completeness and tidyness.
<br>**Either of the two datasets can be used, depending on the analytical task at the hand!**

In [126]:
#Exporting both the datasets to CSV files
df_pitches.to_csv('Pitches_Tidy.csv')
df_pitches_complete.to_csv('Pitches_Complete.csv')